# GCD own algorithm
[GCD detailed script](https://www.mathcha.io/editor/dGXnWtP8twWsGPdy2Ycl5ly7xhd9Xy8ptKleZqr)

In [1]:
import sys
import time

sys.path.append("../")
from calculus.term import *
from calculus.strategy import *
from calculus.advanced_terms import *

GCD≡Y (λf.λn.λm.
$\;\;\;$ ITE (OR (ISZERO n) (ISZERO m)) [1]
$\;\;\;\;\;\;$ (ITE (EQ n m) n
$\;\;\;\;\;\;$ (ITE (LE n m) (f n (SUBSTR m n)) (f m (SUBSTR n m))))))

GCD = Y ( λf.λn.λm.
            (ITE (OR (ISZERO n) (ISZERO m) )
                [1]
                (ITE (EQ n m)
                    n
                    (ITE (LE n m)
                        (f n (SUBSTR m n))
                        (f m (SUBSTR n m))
                     )
                 )
             )
         )



In [3]:
def gcd_term_v0():
    n, m, f = Var(), Var(), Var()
    n_, m_, f_ = Atom(n), Atom(m), Atom(f)

    swap_line = multi_app_term(
        ite_term(),
        multi_app_term(le_term(), n_, m_),
        multi_app_term(f_, m_, multi_app_term(substr_term(), n_, m_)),
        multi_app_term(f_, n_, multi_app_term(substr_term(), m_, n_)),
    )

    eq_nm_line = multi_app_term(ite_term(), multi_app_term(eq_term(), n_, m_), n_, swap_line)
    zero_check = multi_app_term(or_term(), App(iszero_term(), n_), App(iszero_term(), m_))

    inner_app = multi_app_term(ite_term(), zero_check, n_term(1), eq_nm_line)
    inner_lambda = Lambda(f, Lambda(n, Lambda(m, inner_app)))
    return App(y_term(), inner_lambda)

In [4]:
def test_gcd_v0(var_0=0, var_1=0, strategy=LeftmostOutermostStrategy()):
    start_time = time.time()
    term_ = multi_app_term(gcd_term_v0(), n_term(var_0), n_term(var_1))
    res_term, steps = term_.normalize_no_lim(strategy)

    # print(term_, "\n\n")
    print(f"Norm steps: {steps}")
    print(res_term)
    print(f"time normalization: {time.time() - start_time}s")

In [4]:
test_gcd_v0(0, 0, LeftmostOutermostStrategy())

KeyboardInterrupt: 

In [5]:
test_gcd_v0(1, 1, LeftmostOutermostStrategy())

KeyboardInterrupt: 

In [6]:
test_gcd_v0(0, 0, RightmostOutermostStrategy())

KeyboardInterrupt: 

# test the v00 realization

In [7]:
def gcd_term_v00():
    n, m, f = Var(), Var(), Var()
    n_, m_, f_ = Atom(n), Atom(m), Atom(f)
    sub_m_n = App(App(substr_term(), m_), n_)
    sub_n_m = App(App(substr_term(), n_), m_)
    iszero_n = App(iszero_term(), n_)
    iszero_m = App(iszero_term(), m_)
    f_n_sub = App(App(f_, n_), sub_m_n)
    f_m_sub = App(App(f_, m_), sub_n_m)
    le_n_m = App(App(le_term(), n_), m_)
    eq_n_m = App(App(eq_term(), n_), m_)
    or_is_zero = App(or_term(), App(iszero_n, iszero_m))
    ite_le_sub_sub = App(App(App(ite_term(), le_n_m), f_n_sub), f_m_sub)
    ite_eq_ite = App(App(App(ite_term(), eq_n_m), n_), ite_le_sub_sub)
    or_is_ite = App(App(or_is_zero, n0_term()), ite_eq_ite)
    inner_app = App(ite_term(), or_is_ite)
    inner_lambda = Lambda(f, Lambda(n, Lambda(m, inner_app)))
    return App(y_term(), inner_lambda)


def test_gcd_v00(var_0=0, var_1=0, strategy=LeftmostOutermostStrategy()):
    start_time = time.time()
    term_ = multi_app_term(gcd_term_v00(), n_term(var_0), n_term(var_1))
    res_term, steps = term_.normalize_no_lim(strategy)

    # print(term_, "\n\n")
    print(f"Norm steps: {steps}")
    print(res_term)
    print(f"time normalization: {time.time() - start_time}s")

In [8]:
test_gcd_v00(0, 0, LeftmostOutermostStrategy())

Norm steps: 104
λq.λw.w
time normalization: 4.177680253982544s


In [9]:
test_gcd_v00(0, 1, LeftmostOutermostStrategy())

KeyboardInterrupt: 

In [10]:
test_gcd_v00(1, 0, LeftmostOutermostStrategy())

Norm steps: 42
λw.λq.q
time normalization: 1.4063191413879395s


In [11]:
test_gcd_v00(1, 1, LeftmostOutermostStrategy())

Norm steps: 69
λw.λq.(w q)
time normalization: 2.261972427368164s


In [12]:
test_gcd_v00(4, 2, LeftmostOutermostStrategy())

Norm steps: 96
λw.λq.(w (w q))
time normalization: 2.687309980392456s


In [13]:
test_gcd_v00(2, 4, LeftmostOutermostStrategy())

Norm steps: 150
λq.λw.(q (q (q (q w))))
time normalization: 3.6586380004882812s


In [14]:
test_gcd_v00(10, 5, LeftmostOutermostStrategy())

Norm steps: 177
λw.λq.(w (w (w (w (w q)))))
time normalization: 5.694246768951416s


In [15]:
test_gcd_v00(5, 10, LeftmostOutermostStrategy())

Norm steps: 312
λw.λq.(w (w (w (w (w (w (w (w (w (w q))))))))))
time normalization: 25.131871223449707s


# test the v01 realization

In [16]:
def gcd_term_v01():
    n, m, f = Var(), Var(), Var()
    n_, m_, f_ = Atom(n), Atom(m), Atom(f)
    sub_m_n = App(App(substr_term(), m_), n_)
    sub_n_m = App(App(substr_term(), n_), m_)
    iszero_n = App(iszero_term(), n_)
    iszero_m = App(iszero_term(), m_)
    f_n_sub = App(App(f_, n_), sub_m_n)
    f_m_sub = App(App(f_, m_), sub_n_m)
    le_n_m = App(App(le_term(), n_), m_)
    eq_n_m = App(App(eq_term(), n_), m_)
    or_is_zero = App(or_term(), App(iszero_n, iszero_m))
    ite_le_sub_sub = App(App(App(ite_term(), le_n_m), f_n_sub), f_m_sub)
    ite_eq_ite = App(App(App(ite_term(), eq_n_m), n_), ite_le_sub_sub)
    # or_is_ite = App(App(or_is_zero, n0_term()), ite_eq_ite)
    # inner_app = App(ite_term(), or_is_ite)
    inner_app = App(App(App(ite_term(), or_is_zero), n0_term()), ite_eq_ite)
    inner_lambda = Lambda(f, Lambda(n, Lambda(m, inner_app)))
    return App(y_term(), inner_lambda)


def test_gcd_v01(var_0=0, var_1=0, strategy=LeftmostOutermostStrategy()):
    start_time = time.time()
    term_ = multi_app_term(gcd_term_v01(), n_term(var_0), n_term(var_1))
    res_term, steps = term_.normalize_no_lim(strategy)

    # print(term_, "\n\n")
    print(f"Norm steps: {steps}")
    print(res_term)
    print(f"time normalization: {time.time() - start_time}s")

In [17]:
test_gcd_v01(0, 0, LeftmostOutermostStrategy())

KeyboardInterrupt: 

In [18]:
test_gcd_v01(0, 1, LeftmostOutermostStrategy())

KeyboardInterrupt: 

In [19]:
test_gcd_v01(1, 0, LeftmostOutermostStrategy())

KeyboardInterrupt: 

In [20]:
test_gcd_v01(5, 10, LeftmostOutermostStrategy())

KeyboardInterrupt: 

In [21]:
test_gcd_v01(10, 0, LeftmostOutermostStrategy())

KeyboardInterrupt: 